Daq Correlator Demo

Start by importing all the modules to be used and setting the TANGO_HOST environment variable.

In [ ]:
import tango
import time
import json
import os

os.environ["TANGO_HOST"] = "tango-databaseds-test.ska-low-mccs-spshw:10000"

Add a helper method to poll devices

In [ ]:
def poll_until_state(device: tango.DeviceProxy, state: tango._tango.DevState) -> None:
    """Function to poll device until in wanted state."""
    while device.state() != state:
        time.sleep(1)

Set up device proxies for our tango devices and put them into `adminMode.ONLINE` (allowing them to be used)

In [ ]:
# create device proxies and put devices in Online, not simulated mode
subrack=tango.DeviceProxy("low-mccs/subrack/0001")
subrack.adminMode=0

tile=tango.DeviceProxy("low-mccs/tile/0002")
tile.adminMode = 0

daq = tango.DeviceProxy("low-mccs/daqreceiver/001")
daq.adminMode = 0
poll_until_state(daq, tango._tango.DevState.ON) # Daq is an always-on device.

# turn on subrack and wait for ON status
subrack.On()
poll_until_state(subrack, tango._tango.DevState.ON)

# turn on TPM and wait for initialization. 
tile.On()
while tile.tileprogrammingstate != "Initialised" and \
	tile.tileprogrammingstate != "Synchronised":
	time.sleep(1)

Configure the DaqReceiver to receive data.

In [ ]:
daq.SetConsumers("CORRELATOR_DATA")
daq_config = {
    "nof_antennas": 16,
    "nof_tiles": 1,
    "acquisition_duration": 30,
    "receiver_frame_size": 9000,
}
daq.Configure(json.dumps(daq_config))

Set up the data stream on the tile and daq

In [ ]:
# Start TPM acquisition and wait for ADC stream to start
tile.startacquisition('{}')
while tile.tileprogrammingstate != "Synchronised":
	time.sleep(1)

#configure networking between TPM and DAQ.
dst_tpm_port = 4660
dst_tpm_ip = "10.0.10.210" # TODO: Get DAQ IP address.
lmc_download_dict = {"mode": "10g", "destination_ip": dst_tpm_ip, "destination_port": dst_tpm_port}
tile.SetLmcDownload(json.dumps(lmc_download_dict))

Start the daq

In [ ]:
daq.Start("") # Start DAQ with currently configured "default" consumer(s).

data_to_send = {"data_type": "channel_continuous", "channel_id": 1}
tile.SendDataSamples(json.dumps(data_to_send))
# Daq receives data here?
# Files appear?
# How do we "prove" that we have correlated data? Just that?